In [ ]:
from nlb_tools.nwb_interface import NWBDataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import math

from Area2_analysis.funcs import angle_between, pred_with_new_weights
from Area2_analysis.funcs import nans, fit_and_predict
from Area2_analysis.funcs import sub_and_predict

In [ ]:
foldername = "~/area2_population_analysis/s1-kinematics/actpas_NWB/"
monkey = "Han_20171207"
filename = foldername + monkey + "_COactpas_TD.nwb"

dataset_5ms = NWBDataset(filename, split_heldout=False)
xy_vel = dataset_5ms.data['hand_vel'].to_numpy()
xy_acc = np.diff(xy_vel, axis = 0, prepend=[xy_vel[0]])
dataset_5ms.add_continuous_data(xy_acc,'hand_acc',chan_names = ['x','y'])

dataset_5ms.resample(5)
dataset_5ms.smooth_spk(40, name='smth_40')
bin_width = dataset_5ms.bin_width
print(bin_width)

In [ ]:
n_dims = 20 # for PCA

active_mask = (~dataset_5ms.trial_info.ctr_hold_bump) & (dataset_5ms.trial_info.split != 'none')
passive_mask = (dataset_5ms.trial_info.ctr_hold_bump) & (dataset_5ms.trial_info.split != 'none')

n_neurons = dataset_5ms.data.spikes.shape[1]
print(n_neurons,'neurons')

all_data = np.array(dataset_5ms.data.spikes_smth_40)
print(all_data.shape)
data_for_pca = all_data[~np.isnan(all_data).any(axis=1)]
print(data_for_pca.shape)

scaler = StandardScaler()
X = scaler.fit_transform(data_for_pca)
pca = PCA(n_components=n_dims)
X = pca.fit(X)

PCA_data = nans([all_data.shape[0],n_dims])
idx = 0
for dp in all_data:
    dp = dp.reshape((1, -1))
    if np.isnan(dp).any():
        dp_pca = nans([1,n_dims])
    else:
        dp_pca = pca.transform(scaler.transform(dp))
    PCA_data[idx,:] = dp_pca
    idx+=1
print(PCA_data.shape)
dataset_5ms.add_continuous_data(PCA_data,'PCA')
print('PCA total var explained:',sum(pca.explained_variance_ratio_))

In [ ]:
#make dictionary for trial condition (reaching directions) for Stratified CV
trial_mask = active_mask
active_trials_idx = np.array(dataset_5ms.trial_info.loc[trial_mask]['trial_id'])
n_trials = dataset_5ms.trial_info.loc[trial_mask].shape[0]
print(n_trials,'active trials')

cond_dir_idx = []
cond_dict = nans([n_trials])
for direction in [0,45,90,135,180,225,270,315]:
    cond_dir_idx.append(np.where((dataset_5ms.trial_info['cond_dir'] == direction) & (dataset_5ms.trial_info['ctr_hold_bump'] == False) & \
           (dataset_5ms.trial_info['split'] != 'none'))[0])
i = 0
for idx in active_trials_idx:
    for cond in range(0,len(cond_dir_idx)):
        if idx in cond_dir_idx[cond]:
            cond_dict[i] = cond
            break
    i+=1
print(cond_dict)

In [ ]:
# # Passive and active afferent decoders angle
# dataset = dataset_5ms
# x_field = 'PCA'
# y_field ='hand_acc'

# dim = n_dims
# pred_range = (-100,120)
# act_lag_axis = np.arange(0,151,10)
# pas_lag_axis = np.arange(0,151,10)

# act_coef_array = nans([len(act_lag_axis),2,dim])
# pas_coef_array = nans([len(pas_lag_axis),2,dim])

# trial_mask = active_mask
# for i in range(len(act_lag_axis)):
#     lag = act_lag_axis[i]
#     _, coef, _ = fit_and_predict(dataset, trial_mask, 'move_onset_time', pred_range, lag, x_field, y_field)
#     act_coef_array[i,:,:] = coef

# trial_mask = passive_mask
# for i in range(len(pas_lag_axis)):
#     lag = pas_lag_axis[i]
#     _, coef, _ = fit_and_predict(dataset, trial_mask, 'move_onset_time', pred_range, lag, x_field, y_field)
#     pas_coef_array[i,:,:] = coef


# act_t_label = act_lag_axis
# pas_t_label = pas_lag_axis
# act_X_coef_array = act_coef_array[:,0,:]
# pas_X_coef_array = pas_coef_array[:,0,:]
# angDist_array = nans([len(act_t_label),len(pas_t_label)])

# for i in range(len(act_t_label)):
#     for j in range(len(pas_t_label)):
#         angDist_array[i,j] = math.degrees(angle_between(act_X_coef_array[i,:],pas_X_coef_array[j,:]))
# fig, ax = plt.subplots(figsize=(10, 10))
# im = ax.imshow(angDist_array)
# ax.set_xlabel('Passive lag time (ms)')
# ax.set_ylabel('Active lag time (ms)')

# ax.set_xticks(np.arange(len(pas_t_label)))
# ax.set_yticks(np.arange(len(act_t_label)))
# ax.set_xticklabels(labels=pas_t_label)
# ax.set_yticklabels(labels=act_t_label)

# ax.set_title("Angle between decoder weights")
# fig.tight_layout()

# for i in range(len(act_t_label)):
#     for j in range(len(pas_t_label)):
#         text = ax.text(j, i, str(int(angDist_array[i, j])),
#                         ha="center", va="center", color="w")
# plt.tight_layout()

In [ ]:
# Afferent feedback suppression in active trials

dataset = dataset_5ms
plot_dim = 'x'
x_field = 'PCA'
y_field ='hand_acc'
putative_eff_lag = -60
putative_aff_lag = 80
pred_range = (-100,500)

_, aff_axis, _ = fit_and_predict(dataset, active_mask, 'move_onset_time', pred_range, putative_aff_lag, x_field, y_field,cond_dict = cond_dict)
_, eff_axis,_ = sub_and_predict(dataset_5ms, active_mask, 'move_onset_time', pred_range, putative_eff_lag,x_field,y_field,aff_axis, cond_dict = cond_dict)

print(math.degrees(angle_between(aff_axis[0,:],eff_axis[0,:])))

In [ ]:
# # 2D plot
pred_range =(-1000,500)
x_axis = np.arange(pred_range[0], pred_range[1], dataset.bin_width)
aff_axis_x = aff_axis[1,:]
eff_axis_x = eff_axis[1,:]

## Define Passive
trial_mask = passive_mask
df = dataset.make_trial_data(align_field= 'move_onset_time', align_range=pred_range, ignored_trials=~trial_mask)
n_trials = dataset.trial_info.loc[trial_mask].shape[0]
n_timepoints = int((pred_range[1] - pred_range[0])/dataset.bin_width)
aff_proj = np.mean((aff_axis_x @ df.PCA.to_numpy().T).reshape(n_trials, n_timepoints),axis=0)
eff_proj = np.mean((eff_axis_x @ df.PCA.to_numpy().T).reshape(n_trials, n_timepoints),axis=0)

## Plot Passive (1)
fig, ax = plt.subplots()
fig.set_figheight(4)
plt.xlabel('Afferent axis')
plt.ylabel('Efferent axis')
plt.title('')
plt.scatter(aff_proj,eff_proj,s = 10,c=x_axis, cmap='Greys')
plt.scatter(aff_proj[np.argwhere(x_axis==0)],eff_proj[np.argwhere(x_axis==0)],color = 'green',zorder=10)
plt.scatter(aff_proj[np.argwhere(x_axis==120)],eff_proj[np.argwhere(x_axis==120)],color = 'red',zorder=10)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

## Plot Passive (2)
# fig, ax = plt.subplots()
# fig.set_figheight(4)
# plt.xlabel('Efferent axis')
# plt.ylabel('Afferent axis')
# plt.title('')
# plt.scatter(eff_proj,aff_proj,s = 10,c=x_axis, cmap='Greys')
# plt.scatter(eff_proj[np.argwhere(x_axis==0)],aff_proj[np.argwhere(x_axis==0)],color = 'green',zorder=10)
# plt.scatter(eff_proj[np.argwhere(x_axis==120)],aff_proj[np.argwhere(x_axis==120)],color = 'red',zorder=10)
# ax.spines['top'].set_visible(False)
# ax.spines['right'].set_visible(False)


## Define Active
trial_mask = active_mask
df = dataset.make_trial_data(align_field= 'move_onset_time', align_range=pred_range, ignored_trials=~trial_mask)
n_trials = dataset.trial_info.loc[trial_mask].shape[0]
n_timepoints = int((pred_range[1] - pred_range[0])/dataset.bin_width)
aff_proj = np.mean((aff_axis_x @ df.PCA.to_numpy().T).reshape(n_trials, n_timepoints),axis=0)
eff_proj = np.mean((eff_axis_x @ df.PCA.to_numpy().T).reshape(n_trials, n_timepoints),axis=0)

## Plot Active (1)
plt.plot(color = 'copper',label = 'Active proj')
plt.scatter(aff_proj,eff_proj,s = 10,c=x_axis, cmap='Blues')
plt.scatter(aff_proj[np.argwhere(x_axis==0)],eff_proj[np.argwhere(x_axis==0)],color = 'green',zorder=10)
plt.scatter(aff_proj[np.argwhere(x_axis==120)],eff_proj[np.argwhere(x_axis==120)],color = 'red',zorder=10)

## Plot Active (2)
# plt.plot(color = 'copper',label = 'Active proj')
# plt.scatter(eff_proj,aff_proj,s = 10,c=x_axis, cmap='Blues')
# plt.scatter(eff_proj[np.argwhere(x_axis==0)],aff_proj[np.argwhere(x_axis==0)],color = 'green',zorder=10)
# plt.scatter(eff_proj[np.argwhere(x_axis==120)],aff_proj[np.argwhere(x_axis==120)],color = 'red',zorder=10)

from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='#1f77b4', label='Active'),
                    Patch(facecolor='#7f7f7f', label='Passive')]
plt.legend(handles=legend_elements)


In [ ]:
# # 1D plot
pred_range =(-1000,500)
putative_aff_lag = 100
trial_mask = passive_mask
_, pas_aff_axis, _ = fit_and_predict(dataset, trial_mask, 'move_onset_time', (-100,120), putative_aff_lag, x_field, y_field)
x_axis = np.arange(pred_range[0], pred_range[1], dataset.bin_width)

trial_mask = passive_mask
axis = pas_aff_axis[0,:]
df = dataset.make_trial_data(align_field= 'move_onset_time', align_range=pred_range, ignored_trials=~trial_mask)
proj = (axis @ df.PCA.to_numpy().T)
n_trials = dataset.trial_info.loc[trial_mask].shape[0]
n_timepoints = int((pred_range[1] - pred_range[0])/dataset.bin_width)
proj_reshaped = proj.reshape(n_trials, n_timepoints)
mean_proj = np.mean(proj_reshaped,axis = 0)
fig, ax = plt.subplots()
fig.set_figheight(4)
plt.plot(mean_proj,x_axis,color = 'k',label = 'Passive')
plt.xlabel('Projection value')
plt.title('Passive projection')
plt.yticks([])
plt.scatter(mean_proj,x_axis,s = 10,color = 'k')
plt.scatter(mean_proj[np.argwhere(x_axis==0)],0,color = 'green',zorder=10)
plt.scatter(mean_proj[np.argwhere(x_axis==120)],120,color = 'red',zorder=10)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

trial_mask = active_mask
df = dataset.make_trial_data(align_field= 'move_onset_time', align_range=pred_range, ignored_trials=~trial_mask)
proj = (axis @ df.PCA.to_numpy().T)
n_trials = dataset.trial_info.loc[trial_mask].shape[0]
n_timepoints = int((pred_range[1] - pred_range[0])/dataset.bin_width)
proj_reshaped = proj.reshape(n_trials, n_timepoints)
mean_proj = np.mean(proj_reshaped,axis = 0)

plt.plot(mean_proj,x_axis,label = 'Active')
plt.xlabel('Afferent axis')
plt.title('')
plt.scatter(mean_proj,x_axis,s = 10)
plt.scatter(mean_proj[np.argwhere(x_axis==0)],0,color = 'green',zorder=10)
plt.scatter(mean_proj[np.argwhere(x_axis==120)],120,color = 'red',zorder=10)
plt.legend()

In [ ]:
# # Efference copy strength over time (in active)

# plot_dir = [0.0, 90.0, 180.0, 270.0] # limit plot directions to reduce cluttering
# colors = ['red', 'blue', 'green', 'orange']

# positive_lag = 100
# negative_lag = -100

# plot_dim = 'x'
# y_field ='hand_acc'
# fit_range = (0,120)
# pred_range = (-100,500)
# x_axis = np.arange(pred_range[0], pred_range[1], dataset.bin_width)

# _, aff_weights, _ = fit_and_predict(dataset, active_mask, 'move_onset_time', fit_range, positive_lag, x_field, y_field)
# _, eff_weights, _ = fit_and_predict(dataset, active_mask, 'move_onset_time', fit_range, negative_lag, x_field, y_field)
# _, _, aff_vel_df = pred_with_new_weights(dataset, active_mask, 'move_onset_time',pred_range, positive_lag,x_field,y_field, aff_weights)
# _, _, eff_vel_df = pred_with_new_weights(dataset, active_mask, 'move_onset_time',pred_range, negative_lag,x_field,y_field, eff_weights)


# fig, axs = plt.subplots(1, 4, sharex=True, sharey=True, figsize=(16, 4))
# i = 0
# for trial_dir, color in zip(plot_dir, colors):
#     cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
#     for _, trial in eff_vel_df[np.isin(eff_vel_df.trial_id, cond_ids)].groupby('trial_id'):
#         axs[i].plot(x_axis, trial[y_field][plot_dim], color=color, linewidth=0.3)
#         axs[i].plot(x_axis, trial.pred_vel[plot_dim], color='k', alpha = 0.2, linewidth=0.5)
#     for _, trial in aff_vel_df[np.isin(aff_vel_df.trial_id, cond_ids)].groupby('trial_id'):
#         axs[i].plot(x_axis, trial.pred_vel[plot_dim], color='magenta', alpha = 0.2, linewidth=0.5)
    
#     i+=1
# axs[1].set_xlabel('Time (ms)')
# axs[0].set_ylabel('y-vel')
# plt.tight_layout()


In [ ]:
# # Efference copy strength over time (in active)

# plot_dim = 'x'
# y_field ='hand_acc'
# pred_range = (-100,500)
# x_axis = np.arange(pred_range[0], pred_range[1], dataset.bin_width)

# _, _, aff_vel_df = fit_and_predict(dataset, active_mask, 'move_onset_time', pred_range, positive_lag, x_field, y_field)
# _, _, eff_vel_df = fit_and_predict(dataset, active_mask, 'move_onset_time', pred_range, negative_lag, x_field, y_field)


# fig, axs = plt.subplots(1, 4, sharex=True, sharey=True, figsize=(16, 4))
# i = 0
# for trial_dir, color in zip(plot_dir, colors):
#     cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
#     for _, trial in eff_vel_df[np.isin(eff_vel_df.trial_id, cond_ids)].groupby('trial_id'):
#         axs[i].plot(x_axis, trial[y_field][plot_dim], color=color, linewidth=0.3)
#         axs[i].plot(x_axis, trial.pred_vel[plot_dim], color='k', alpha = 0.2, linewidth=0.5)
#     for _, trial in aff_vel_df[np.isin(aff_vel_df.trial_id, cond_ids)].groupby('trial_id'):
#         axs[i].plot(x_axis, trial.pred_vel[plot_dim], color='magenta', alpha = 0.2, linewidth=0.5)
    
#     i+=1
# axs[1].set_xlabel('Time (ms)')
# axs[0].set_ylabel('x-vel')
# plt.tight_layout()
